In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [85]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [86]:
def translateIndex(index, size):
    return (2 * index - size + 1) / size

In [87]:
def transformPHT(image, size, maxOrder):

    t_len = 2 * maxOrder + 1
    
    imvec = np.reshape(np.array(image), (size * size, 1))
    transformed = np.array([])
    
    global T_STEP
    try:
        T_STEP
    except NameError:
        T_STEP =  int(maxOrder / 3) + 1

    # GENERATE MATRIX TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((t_len * t_len, 1))
    vecM = vecM.reshape((t_len * t_len, 1))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((1, size * size))
    vecK = vecK.reshape((1, size * size))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX

    update_progress(0)
    for row in range(int(np.ceil(t_len / T_STEP))):
        start = row * T_STEP * t_len
        end = np.min([row * T_STEP + T_STEP, t_len]) * t_len
        matR = np.repeat(vecR, end - start, axis=0)
        matT = np.repeat(vecT, end - start, axis=0)
        matN = np.repeat(vecN[start:end, :], size * size, axis=1)
        matM = np.repeat(vecM[start:end, :], size * size, axis=1)
        matC = (matR <= 1)
        matF = (np.abs(matN) + np.abs(matM) <= maxOrder)
        matF = matF * 0 + 1

        matW = matF * matC * 4 / (np.pi * size * size) * np.exp(-2 * np.pi * matN * matR * matR * 1j - matM * matT * 1j)
        trf_row = np.matmul(matW, imvec)
        transformed = np.append(transformed, trf_row)

        # FREE UP SOME MEMORY - 2
        del matW
        del matN
        del matM
        del matR
        del matT
        del matC
        del matF

        update_progress((row * T_STEP + T_STEP) / t_len)

    update_progress(1)
    
    transformed2 = np.reshape(transformed, (t_len, t_len))
    del transformed
    return transformed2

In [88]:
def inversePHT(momen, size, maxOrder):
    
    t_len = 2 * maxOrder + 1

    momenvec = np.reshape(np.array(momen), (t_len * t_len, 1))
    inversed = np.array([])
    
    global I_STEP
    try:
        I_STEP
    except NameError:
        I_STEP =  int(size / 6) + 1


    # GENERATE MATRIX INVERS TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((1, t_len * t_len))
    vecM = vecM.reshape((1, t_len * t_len))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((size * size, 1))
    vecK = vecK.reshape((size * size, 1))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX

    update_progress(0)
    for row in range(int(np.ceil(size / I_STEP))):
        start = row * I_STEP * size
        end = np.min([row * I_STEP + I_STEP, size]) * size
        matR = np.repeat(vecR[start:end, :], t_len * t_len, axis=1)
        matT = np.repeat(vecT[start:end, :], t_len * t_len, axis=1)
        matN = np.repeat(vecN, end - start, axis=0)
        matM = np.repeat(vecM, end - start, axis=0)
        matC = (matR <= 1)
        matF = (np.abs(matN) + np.abs(matM) <= maxOrder)
        matF = matF * 0 + 1

        matV = matF * matC * np.exp(2 * np.pi * matN * matR * matR * 1j + matM * matT * 1j)
        inv_row = np.matmul(matV, momenvec)
        inversed = np.append(inversed, inv_row)

        # FREE UP SOME MEMORY - 2
        del matV
        del matN
        del matM
        del matR
        del matT
        del matC
        del matF

        update_progress((row * I_STEP + I_STEP) / size)

    update_progress(1)
    
    inversed2 = np.reshape(inversed, (size, size))
    del inversed
    return inversed2

In [ ]:
def getMatW(size, maxOrder):

    t_len = 2 * maxOrder + 1

    # GENERATE MATRIX TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((t_len * t_len, 1))
    vecM = vecM.reshape((t_len * t_len, 1))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((1, size * size))
    vecK = vecK.reshape((1, size * size))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX
    
    matR = np.repeat(vecR, t_len * t_len, axis=0)
    matT = np.repeat(vecT, t_len * t_len, axis=0)
    matN = np.repeat(vecN, size * size, axis=1)
    matM = np.repeat(vecM, size * size, axis=1)
    
    matW = 4 / (np.pi * size * size) * np.exp(-2 * np.pi * matN * matR * matR * 1j - matM * matT * 1j)
    
    del matN
    del matM
    del matR
    del matT

    return matW

def getMatV(size, maxOrder):
    
    t_len = 2 * maxOrder + 1


    # GENERATE MATRIX INVERS TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((1, t_len * t_len))
    vecM = vecM.reshape((1, t_len * t_len))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((size * size, 1))
    vecK = vecK.reshape((size * size, 1))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX
    
    matR = np.repeat(vecR, t_len * t_len, axis=1)
    matT = np.repeat(vecT, t_len * t_len, axis=1)
    matN = np.repeat(vecN, size * size, axis=0)
    matM = np.repeat(vecM, size * size, axis=0)
    
    matV = np.exp(2 * np.pi * matN * matR * matR * 1j + matM * matT * 1j)
    
    del matN
    del matM
    del matR
    del matT

    return matV

In [ ]:
def calcBE(W1, W2):
    return np.sum(np.abs(np.abs(W1) - np.abs(W2)))